---

# **101 Food Classification**

---

In [ ]:
# !pip install keras==2.15.0

In [ ]:
import keras
keras.__version__

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16
from keras.src.layers.pooling.average_pooling2d import AvgPool2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
from keras.layers import Dense,Input,Flatten
from tensorflow.keras.utils import load_img,img_to_array
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
print(os.listdir("/kaggle/input/food-101/food-101/food-101/images/"))

---

In [ ]:
values = ['macarons', 'french_toast', 'lobster_bisque', 'prime_rib', 'pork_chop', 'guacamole', 'baby_back_ribs', 'mussels', 'beef_carpaccio', 'poutine', 'hot_and_sour_soup', 'seaweed_salad', 'foie_gras', 'dumplings', 'peking_duck', 'takoyaki', 'bibimbap', 'falafel', 'pulled_pork_sandwich', 'lobster_roll_sandwich', 'carrot_cake', 'beet_salad', 'panna_cotta', 'donuts', 'red_velvet_cake', 'grilled_cheese_sandwich', 'cannoli', 'spring_rolls', 'shrimp_and_grits', 'clam_chowder', 'omelette', 'fried_calamari', 'caprese_salad', 'oysters', 'scallops', 'ramen', 'grilled_salmon', 'croque_madame', 'filet_mignon', 'hamburger', 'spaghetti_carbonara', 'miso_soup', 'bread_pudding', 'lasagna', 'crab_cakes', 'cheesecake', 'spaghetti_bolognese', 'cup_cakes', 'creme_brulee', 'waffles', 'fish_and_chips', 'paella', 'macaroni_and_cheese', 'chocolate_mousse', 'ravioli', 'chicken_curry', 'caesar_salad', 'nachos', 'tiramisu', 'frozen_yogurt', 'ice_cream', 'risotto', 'club_sandwich', 'strawberry_shortcake', 'steak', 'churros', 'garlic_bread', 'baklava', 'bruschetta', 'hummus', 'chicken_wings', 'greek_salad', 'tuna_tartare', 'chocolate_cake', 'gyoza', 'eggs_benedict', 'deviled_eggs', 'samosa', 'sushi', 'breakfast_burrito', 'ceviche', 'beef_tartare', 'apple_pie', '.DS_Store', 'huevos_rancheros', 'beignets', 'pizza', 'edamame', 'french_onion_soup', 'hot_dog', 'tacos', 'chicken_quesadilla', 'pho', 'gnocchi', 'pancakes', 'fried_rice', 'cheese_plate', 'onion_rings', 'escargots', 'sashimi', 'pad_thai', 'french_fries']
values.sort()
values = values[1:]
print(values)

In [ ]:
print("Number of classes:",len(values))

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.1)
train_data = train_datagen.flow_from_directory('/kaggle/input/food-101/food-101/food-101/images/',
                                               target_size=(224,224),
                                               batch_size=100,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset='training')
test_data = train_datagen.flow_from_directory('/kaggle/input/food-101/food-101/food-101/images/',
                                               target_size=(224,224),
                                               batch_size=100,
                                               class_mode='categorical',
                                               shuffle=False,
                                               subset='validation')

In [ ]:
print("Images Shape:",train_data.image_shape)

In [ ]:
print('\nBatch Size:',100,
     "\nNunmber of Batches in training set:",len(train_data),
     "\nNunmber of Batches in testing set:",len(test_data),
     "\nNumber of Samples in training set:",train_data.samples,"Samples",
     "\nNumber of Samples in testing set:",test_data.samples,"Samples")

In [ ]:
print("\nThe 101 Classes numbers:\n",np.unique(train_data.labels),"\n",
       "_"*30,
       "\nThe 101 classes names:\n",train_data.class_indices,
      sep="")

In [ ]:
train_data.image_shape

In [ ]:
np.unique(train_data.labels)

In [ ]:
np.unique(test_data.labels)

In [ ]:
train_data.class_indices

---

# Inception V3 Architecture
(Best accuracy)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

x = base_model.output
x = AveragePooling2D()(x)
x = Dropout(.5)(x)
x = Flatten()(x)
x = Dense(101, kernel_initializer='glorot_uniform', kernel_regularizer=l2(.0005), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)
model.summary()

In [ ]:
opt = SGD(lr=.1, momentum=.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.models import load_model
model = load_model("/kaggle/input/food-101-model/tensorflow2/food-101/1/model_food_101.h5")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
results = model.fit(train_data, epochs=10, validation_data=test_data,
                  steps_per_epoch=len(train_data), validation_steps=len(test_data),
                  callbacks = EarlyStopping(patience=2, monitor='val_accuracy',restore_best_weights=True)
                  )

In [ ]:
loss, acc = model.evaluate(test_data)

In [ ]:
print("Test Accuracy:",round(acc*100,2),"%","\nTest Loss:",loss)

Training again

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
results = model.fit(train_data, epochs=10, validation_data=test_data,
                  steps_per_epoch=len(train_data), validation_steps=len(test_data),
                  callbacks = EarlyStopping(patience=2, monitor='val_accuracy',restore_best_weights=True)
                  )

In [ ]:
loss, acc = model.evaluate(test_data)

In [ ]:
print("Test Accuracy:",round(acc*100,2),"%","\nTest Loss:",round(loss,4))

In [ ]:
results = {"accuracy":    [0.9924,0.9962,0.9970,0.9986,0.9995,0.9997,0.9997,0.9999,0.9999,1.0000,0.9999,1.0000,1.0000],
           "loss":        [0.0816,0.0646,0.0562,0.0460,0.0379,0.0325,0.0284,0.0245,0.0216,0.0191,0.0173,0.0158,0.0145],
           "val_accuracy":[0.7065,0.7417,0.7487,0.7535,0.7697,0.7693,0.7716,0.7722,0.7721,0.7736,0.7749,0.7743,0.7733],
           "val_loss":    [1.6280,1.4801,1.4059,1.3613,1.2399,1.2145,1.1774,1.1357,1.1133,1.0886,1.0805,1.0645,1.0611]}

In [ ]:
model.save("model_food_101.h5")

In [ ]:
results['val_accuracy']

In [ ]:
results['val_loss']

In [ ]:
i = results['val_loss'].index(round(loss,4))
i

In [ ]:
results['val_accuracy'][i]

**Visualize training history**

In [ ]:
fig = plt.figure()
plt.plot(results['accuracy'], c='blue', label='accuracy')
plt.plot(results['loss'], c='red', label='loss')
plt.scatter(i,results['accuracy'][i], c='orange', marker='o', label='choosen parameters')
plt.title('Training data')
plt.legend(loc='upper right')
plt.show()

**Visualize testing history**

In [ ]:
fig = plt.figure()
plt.plot(results.history['val_accuracy'], c='blue', label='val accuracy')
plt.plot(results.history['val_loss'], c='red', label='val loss')
plt.scatter(i,results.history['val_accuracy'][i], c='orange', marker='o', label='choosen parameters')
plt.title('Testing data')
plt.legend(loc='upper right')
plt.show()

In [ ]:
yp = model.predict(test_data).argmax(axis=1).reshape(-1,)
yp.shape

In [ ]:
m = pd.crosstab(test_data.labels, yp, rownames=['Actual'], colnames=['Predicted'])
m

In [ ]:
print("Heatmap\n")
plt.figure(figsize=(30, 30))
sn.heatmap(m,annot=True, cmap='Blues')
plt.show()

# Testing Samples

In [ ]:
print(os.listdir("/kaggle/input/food-101/food-101/food-101/images/"))

In [ ]:
print("Macarons Sample")
macarons = load_img("/kaggle/input/food-101/food-101/food-101/images/macarons/2428554.jpg",target_size=(224,224))
macarons

In [ ]:
print("Pizza Sample")
pizza = load_img("/kaggle/input/food-101/food-101/food-101/images/pizza/768276.jpg",target_size=(224,224,3))
pizza

In [ ]:
print("Donuts Sample")
donuts = load_img("/kaggle/input/food-101/food-101/food-101/images/donuts/2563686.jpg",target_size=(224,224,3))
donuts

In [ ]:
print("Frensh Toast Sample")
toast = load_img("/kaggle/input/food-101/food-101/food-101/images/french_toast/2769309.jpg",target_size=(224,224,3))
toast

In [ ]:
print("French_fries Sample")
fries = load_img("/kaggle/input/food-101/food-101/food-101/images/french_fries/2246621.jpg",target_size=(224,224))
fries

In [ ]:
print("Ice Cream Sample")
ice = load_img("/kaggle/input/food-101/food-101/food-101/images/ice_cream/579407.jpg",target_size=(224,224))
ice

In [ ]:
fig = plt.figure(figsize=(14,7))
# plt.grid=False
fig.add_subplot(1, 6, 1)
plt.axis('off')
plt.imshow(macarons)
plt.title("Macarons")
# _____________________________________________
fig.add_subplot(1, 6, 2)
plt.axis('off')
plt.imshow(fries)
plt.title("Frensh Fries")
# _____________________________________________
fig.add_subplot(1, 6, 3)
plt.axis('off')
plt.imshow(ice)
plt.title("Ice Cream")
# _____________________________________________
fig.add_subplot(1, 6, 4)
plt.axis('off')
plt.imshow(toast)
plt.title("Frensh Toast")
# _____________________________________________
fig.add_subplot(1, 6, 5)
plt.axis('off')
plt.imshow(pizza)
plt.title("Pizza")
# _____________________________________________
fig.add_subplot(1, 6, 6)
plt.axis('off')
plt.imshow(donuts)
plt.title("Donuts")

In [ ]:
macarons = img_to_array(macarons)
fries = img_to_array(fries)
ice = img_to_array(ice)
pizza = img_to_array(pizza)
donuts = img_to_array(donuts)
toast = img_to_array(toast)
macarons = macarons/255
fries = fries/255
ice = ice/255
pizza = pizza/255
donuts = donuts/255
toast = toast/255
macarons = macarons.reshape(1,224,224,3)
fries = fries.reshape(1,224,224,3)
ice = ice.reshape(1,224,224,3)
pizza = pizza.reshape(1,224,224,3)
donuts = donuts.reshape(1,224,224,3)
toast = toast.reshape(1,224,224,3)

macarons.shape

# Samples Predicting

In [ ]:
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
main_folder_path = '/kaggle/input/food41/images'
food_classes = [name for name in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, name))]
random_food_classes = random.sample(food_classes, 10)
print("List of random food classes:", random_food_classes)

overall_total_images = 0
overall_correct_predictions = 0

for food_class in random_food_classes:
    folder_path = os.path.join(main_folder_path, food_class)
    sample_images = random.sample(os.listdir(folder_path), min(100, len(os.listdir(folder_path))))
    target_class_index = values.index(food_class)
    class_total_images = 0
    class_correct_predictions = 0

    for img_name in sample_images:
        img_path = os.path.join(folder_path, img_name)
        img = load_img(img_path, target_size=(224, 224))
        img_array = img_to_array(img) / 255.0
        img_array = img_array.reshape(1, 224, 224, 3)

        predicted_class_index = model.predict(img_array, verbose=0).argmax()

        if predicted_class_index == target_class_index:
            class_correct_predictions += 1

        class_total_images += 1

    class_accuracy = class_correct_predictions / class_total_images * 100
    print(f"Accuracy for class '{food_class}': {class_accuracy:.2f}%")
    print(f"Total images tested for '{food_class}': {class_total_images}")
    print(f"Correct predictions for '{food_class}': {class_correct_predictions}\n")

    overall_total_images += class_total_images
    overall_correct_predictions += class_correct_predictions

overall_accuracy = overall_correct_predictions / overall_total_images * 100
print(f"\nOverall accuracy on 10 random food classes: {overall_accuracy:.2f}%")
print(f"Overall total images tested: {overall_total_images}")
print(f"Overall correct predictions: {overall_correct_predictions}")


In [ ]:
folder_path = '/kaggle/input/food-101/food-101/food-101/images/pho'
target_class = "pho"
target_class_index = values.index(target_class)
all_images = os.listdir(folder_path)
sample_images = random.sample(all_images, min(100, len(all_images)))
total_images = 0
correct_predictions = 0

sfor img_name in sample_images:
    img_path = os.path.join(folder_path, img_name)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = img_array.reshape(1, 224, 224, 3)

    predicted_class_index = model.predict(img_array, verbose=0).argmax()

    if predicted_class_index == target_class_index:
        correct_predictions += 1

    total_images += 1

accuracy = correct_predictions / total_images * 100
print(f"Accuracy for class '{target_class}' on 100 random images: {accuracy:.2f}%")
print(f"Total images tested: {total_images}")
print(f"Correct predictions: {correct_predictions}")

In [ ]:
a_food_test = load_img("/kaggle/input/food41/images/apple_pie/1112838.jpg",target_size=(224,224))
a_food_test

a_food_test = img_to_array(a_food_test)
a_food_test = a_food_test/255
a_food_test = a_food_test.reshape(1,224,224,3)

p0 = (model.predict(a_food_test)).argmax()
print("Class ",p0,": ",values[p0],sep='')

In [ ]:
p1 = (model.predict(macarons)).argmax()

print("Class ",p1,": ",values[p1],sep='')
print(calories[p1],'\nNote:',s)

In [ ]:
p2 = (model.predict(fries)).argmax()

print("Class ",p2,": ",values[p2],sep='')
print(calories[p2],'\nNote:',s)

In [ ]:
p3 = (model.predict(ice)).argmax()

print("Class ",p3,": ",values[p3],sep='')
print(calories[p3],'\nNote:',s)

In [ ]:
p4 = (model.predict(pizza)).argmax()

print("Class ",p4,": ",values[p4],sep='')
print(calories[p4],'\nNote:',s)

In [ ]:
p5 = (model.predict(donuts)).argmax()

print("Class ",p5,": ",values[p5],sep='')
print(calories[p5],'\nNote:',s)

In [ ]:
p6 = (model.predict(toast)).argmax()

print("Class ",p6,": ",values[p6],sep='')
print(calories[p6],'\nNote:',s)